In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import numpy as np
from sympy import *

from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

## volcanic eruptions and climate change

In [3]:
# stefan-boltzmann constant
sigma_b = 5.6696e-8 # Wm^(-2)K^(-4)

In [4]:
# todo: put regional parameters in config files

# regional parameters
# tau = 1 # atmospheric transmissivity
# alpha_sky = np.ndarray()  # top of atmosphere albedos
# alpha_surface = np.ndarray()  # surface albedos
# gamma = np.ndarray()  # spatial radiation factors
# rho = np.ndarray()  # densities
# epsilon = np.ndarray()  # emissivities
# specific_heats = np.ndarray()  # specific heat capacities (c_k)


#### regional temperature variation

In [32]:
t, phi = symbols('t varphi')
T = Function("T_k")(t, phi)
S = Function("S")(t)
d_temp = T.diff(t)
c, A, Z = symbols("c_k A_k Z_k")
alpha, gamma, rho, tau, sigma, epsilon = symbols("alpha_k gamma_k rho_k tau_k sigma_B varepsilon")

T, d_temp
alpha, gamma, rho, tau, sigma, epsilon

⎛               ∂                ⎞
⎜Tₖ(t, varphi), ──(Tₖ(t, varphi))⎟
⎝               ∂t               ⎠

(αₖ, γₖ, ρₖ, τₖ, σ_B, varepsilon)

In [30]:
diff_eq = Eq(c*rho*Z*d_temp, gamma*(1 - alpha)*S - epsilon*sigma*T**4)
diff_eq

         ∂                             4                              
Zₖ⋅cₖ⋅ρₖ⋅──(Tₖ(t, varphi)) = - ε⋅σ_B⋅Tₖ (t, varphi) + γₖ⋅(1 - αₖ)⋅S(t)
         ∂t                                                           

In [40]:
def d_temperature_dt(time: float, temperatures: np.ndarray, transfer_coefficients: np.ndarray, volcano_model=None, kwargs=None):
    """
    :param time: scalar
    :param temperatures: temperature field
    :param transfer_coefficients: pre-computed heat transfer coefficients
    :param volcano_model: TBD
    :param kwargs: for scipy.integrators.solve_ivp
    :return: Temperatures
    """
    # numerical solve equation 8
    dtemperature = None  # solve regions together

    return dtemperature